In [ ]:
Solutions to SQL Join Interview Questions



In [ ]:
1. Fundamentals (Beginner/Screening)
Definition & Difference:
The four main types of SQL Joins are:

* INNER JOIN: Returns records that have matching values in both tables.

* LEFT JOIN (or LEFT OUTER JOIN): Returns all records from the left table, and the matched records from the right table. If there is no match, the result is NULL from the right side.

* RIGHT JOIN (or RIGHT OUTER JOIN): Returns all records from the right table, and the matched records from the left table. If there is no match, the result is NULL from the left side.

* FULL OUTER JOIN: Returns all records when there is a match in either the left or the right table. If there is no match, it fills the missing side with NULLs.

The primary difference between a LEFT JOIN and a RIGHT JOIN is which table dictates the output rows (the preserved table).     
A LEFT JOIN preserves all rows from the first (left) table, while a RIGHT JOIN preserves all rows from the second (right) table.

Use Case: INNER JOIN:
An INNER JOIN is appropriate when you only need records where a relationship must exist in both tables. It guarantees data integrity 
based on the join key.

Example: To find the names of employees who have an assigned department.

SELECT
    E.EmployeeName,
    D.DepartmentName
FROM
    Employees AS E
INNER JOIN
    Departments AS D ON E.DepartmentID = D.DepartmentID;


Use Case: FULL OUTER JOIN:
You would use a FULL OUTER JOIN when you need to see all records from both tables, regardless of whether a match exists. This is often used for data reconciliation or comparison.

Example: Comparing two customer lists (e.g., "Active Subscriptions" and "Newsletter Signups") to see who is in both, only in the first, or only in the second.

Results for non-matching rows: If a row from the left table has no match in the right table, the columns from the right table will contain NULL. Conversely, if a row from the right table has no match in the left, the columns from the left table will contain NULL.

Implicit vs. Explicit:

Explicit: Uses the JOIN keyword and the ON clause to specify the relationship.

SELECT * FROM TableA A JOIN TableB B ON A.Key = B.Key;


Implicit: Uses a comma between tables in the FROM clause, and the join condition is placed in the WHERE clause.

SELECT * FROM TableA A, TableB B WHERE A.Key = B.Key;


The explicit method is preferred because it separates the join logic from the filtering logic. This makes the query much easier to read, maintain, and less prone to accidental Cartesian Products (if the join condition is accidentally omitted in the WHERE clause).

NULL Values in Join Keys:
A standard INNER JOIN will not match NULL values. This is because, in SQL, NULL is not a value; it represents an unknown state. 
Therefore, the comparison NULL = NULL evaluates to UNKNOWN, not TRUE, and the join fails.
To explicitly match NULLs, you would need a more complex WHERE or ON clause using the IS NULL condition, 
often for data quality checks (though this is rare for primary joins).

In [ ]:
2. Practical Application & Syntax (Intermediate)

LEFT Join Caveat:
Adding a WHERE clause filter on a non-key column from the right table (TableB) effectively converts the LEFT JOIN into an INNER JOIN.

Reason: The purpose of the LEFT JOIN is to preserve rows from the left table, setting right-side columns to NULL when no match exists. If you add WHERE TableB.ColumnX IS NOT NULL (or any filter requiring a value), you are explicitly filtering out the NULL rows generated by the LEFT JOIN, leaving only those rows that had a match in both tables—the definition of an INNER JOIN.

Self-Join:
A Self-Join is a regular join where a table is joined to itself. 
It requires aliasing the table to distinguish between the two roles the table plays in the query.

Scenario: Find all pairs of employees who work in the same city.

SELECT
    A.EmployeeName AS Employee1,
    B.EmployeeName AS Employee2,
    A.City
FROM
    Employees AS A
INNER JOIN
    Employees AS B ON A.City = B.City  -- Join condition
                  AND A.EmployeeID < B.EmployeeID; -- Prevents duplicate pairs (A,B) and (B,A) and self-match (A,A)


Non-Equi Join:
A Non-Equi Join is any join that uses a comparison operator other than the equals sign (=) in the ON or WHERE clause (e.g., <, >, <=, BETWEEN).

Example Scenario: Classifying employees into salary tiers defined in a separate Tiers table.

SELECT
    E.EmployeeName,
    E.Salary,
    T.TierName
FROM
    Employees AS E
INNER JOIN
    SalaryTiers AS T ON E.Salary BETWEEN T.MinSalary AND T.MaxSalary;


Filtering Non-Matches (Anti-Join):
This pattern is known as an Anti-Join.

SELECT
    U.CustomerID,
    U.CustomerName
FROM
    Customers AS U
LEFT JOIN
    Orders AS O ON U.CustomerID = O.CustomerID
WHERE
    O.OrderID IS NULL; -- The crucial element


This pattern is based on the LEFT JOIN. The crucial element is the WHERE O.OrderID IS NULL, which filters the result set to only include the rows from the left table (Customers) that failed to find a match in the right table (Orders), thus having NULL values for the right table's primary key.

Cross Join:
The result of a CROSS JOIN is the Cartesian Product of the two tables. This means every row from the first table is combined with every row from the second table. If Table A has $N$ rows and Table B has $M$ rows, the result has $N \times M$ rows.

Use Case: Generating all possible combinations, such as creating a time series for all products (joining a list of products to a list of dates).

3. Advanced Concepts & Performance (Expert/Data Engineering)

Performance of JOINs:
The two primary algorithms are:

Hash Join: The database builds a hash table on the smaller table (the build side) in memory. It then streams the larger table (the probe side) and probes the hash table for matches. This is usually the fastest method.

Sort-Merge Join: Both tables are first sorted based on the join key. Then, the sorted tables are scanned simultaneously to find matches (like merging two sorted lists).

Bonus: The Hash Join is generally faster for very large, unsorted datasets because it avoids the costly step of sorting the entire dataset, provided the smaller table fits into memory.

Broadcast Join (Spark/Data Warehousing):
A Broadcast Join is a highly optimized Hash Join where the entire smaller table (the build side) is copied (broadcasted) to every single worker node that holds partitions of the larger table.

Efficiency: It is extremely efficient because it eliminates the need for a shuffle (network-intensive data redistribution) of the large table, as all necessary join data is locally available on the worker node.

Spark's Decision: Spark typically decides to perform a Broadcast Join automatically if the smaller DataFrame's size is below a configurable threshold (default is usually 10MB or 25MB, depending on the environment). You can also force it using the broadcast() function in PySpark.

Join Order:
The order of joins is critical for performance, especially with INNER JOINs, as the database optimizer needs to decide the sequence.

General Guideline: Join tables in an order that most rapidly reduces the size of the intermediate result set.

Rule of Thumb: Start by joining the largest table (Fact table) with the smallest, most highly filtered dimension tables first. Filtering small tables early and joining them to the large table first significantly reduces the amount of data that needs to be processed in subsequent, potentially more expensive joins.

Cartesian Product Risk:
A Cartesian Product occurs when you join two tables without specifying a join condition, or when the join condition is always true. It is a mathematical multiplication where every row in the first table is matched with every row in the second table.

Accidental Creation: The most common way is using an implicit join and forgetting the WHERE clause filter (SELECT * FROM TableA, TableB;).

Consequence: It is a performance sink. The resulting dataset size explodes exponentially, consuming all memory and processing time, often crashing the query or cluster.

Star Schema Joins (Fact vs. Dimension):
In a Star Schema, the central Fact Table contains measures and foreign keys. The surrounding Dimension Tables contain descriptive attributes.

The typical join pattern is a series of INNER JOINs (or occasionally LEFT JOINs to preserve facts) connecting the Fact Table's foreign keys to the Dimension Table's primary keys. This is always a 1:N relationship (One Dimension row matches Many Fact rows).

SELECT
    D.Date,
    P.ProductName,
    SUM(F.SalesAmount)
FROM
    FactSales AS F
INNER JOIN
    DimProduct AS P ON F.ProductID = P.ProductID
INNER JOIN
    DimDate AS D ON F.DateID = D.DateID
GROUP BY 1, 2;


4. Coding Challenge

Coding Challenge: Latest Activity:
The most robust solution uses a LEFT JOIN combined with a Window Function (ROW_NUMBER) to rank the orders for each user.

WITH RankedOrders AS (
    SELECT
        user_id,
        order_date,
        -- Rank orders for each user, ordering by date descending (most recent = rank 1)
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY order_date DESC) as rn
    FROM
        Orders
)
SELECT
    U.user_id,
    U.name,
    R.order_date
FROM
    Users AS U
LEFT JOIN
    RankedOrders AS R ON U.user_id = R.user_id AND R.rn = 1; -- Crucial filter: only take the latest order (rank 1)